In [9]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

pd.set_option("display.max_columns", None)

In [3]:
pref = "D:/디지털혁신팀/태블로/CS/CS 현황_FINAL/20210731/"
filenames = ["20년 AS.csv", "21년 AS.csv"]
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
as_list = pd.concat(dfs, ignore_index=True)
# as_list.to_csv(f"{pref}AS리스트.csv", encoding="euc-kr")

filenames = ["19년 BS.csv", "20년 BS.csv", "21년 BS.csv"]
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
bs_list = pd.concat(dfs, ignore_index=True)
bs_list = bs_list.rename({"아이앤콘스":"현장: 아이앤콘스"}, axis=1)

# bs_list.to_csv(f"{pref}BS리스트.csv", encoding="euc-kr")

C:\Users\00006363\Anaconda3\envs\tf2.3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


FileNotFoundError: [Errno 2] No such file or directory: 'D:/디지털혁신팀/태블로/CS/CS 현황_FINAL/20210731/19년 BS.csv'

In [16]:
data = pd.read_csv("D:\Github\Work\Tableau\Sales\사업별미수현황총괄표_모든 사업지.csv", encoding="euc-kr")

In [17]:
data.sample(5)

,코드,내용구분,현황,사업지,분양구분,분양종류,분양일,완공예정,공동도급지분율,사업규모세대,사업규모금액,분양세대,분양금액,미분양세대,미분양금액,임대세대,임대금액,임대분양가,약정금액당월,약정금액누계,입금도래당월,입금도래누계,입금선납당월,입금선납누계,입금누계,미수금기간도래,미수금미도래,미수금계,주간사여부
363,A599,NaN,NaN,대치동(대치도곡주공2차),조합,APT,2005-06-22,2007-07-27,0.0,610,114298770,609,114298770,1,0,0,0,0,0.0,114298770.0,0.0,114298770.0,0.0,0.0,114298770.0,0.0,0.0,0.0,NaN
232,A425,NaN,NaN,수원망포2차,일반,APT,2000-09-19,2002-10-30,0.0,489,88964700,489,88964700,0,0,0,0,0,0.0,88964700.0,0.0,88964700.0,0.0,0.0,88964700.0,0.0,0.0,0.0,NaN
29,A-1,NaN,NaN,운정 IPARK 더테라스 근린생활시설,일반,상가,2020-09-23,2022-07-31,0.0,11,8144700,11,8144700,0,0,0,0,0,0.0,1628940.0,0.0,720270.0,0.0,0.0,720270.0,908670.0,6515760.0,7424430.0,NaN
400,A634,NaN,NaN,잠실1단지,조합,상가,2005-07-20,2008-09-30,0.0,319,21268664,319,21268664,0,0,0,0,0,0.0,21268664.0,0.0,21268664.0,0.0,0.0,21268664.0,0.0,0.0,0.0,NaN
204,A406,NaN,NaN,부산 화명2차,일반,상가,1999-12-21,2001-10-31,0.0,13,1175000,13,1175000,0,0,0,0,0,0.0,1175000.0,0.0,1175000.0,0.0,0.0,1175000.0,0.0,0.0,0.0,NaN
